# 6차시: 성능 최적화 & 모니터링

## 학습 목표
- 응답 캐싱 시스템 구현
- 비동기 처리를 통한 성능 향상
- 토큰 사용량 최적화 전략
- 실시간 모니터링 및 메트릭 수집
- 성능 분석 및 대시보드 구축

## 목차
1. [캐싱 시스템](#캐싱-시스템)
2. [비동기 처리](#비동기-처리)
3. [토큰 최적화](#토큰-최적화)
4. [모니터링 시스템](#모니터링-시스템)
5. [성능 분석](#성능-분석)
6. [실습 과제](#실습-과제)

## 환경 설정

In [ ]:
#!/usr/bin/env python3
"""
6차시: 성능 최적화 & 모니터링
Author: AI Chatbot Workshop
Date: 2024-08-30
Description: 응답 캐싱, 비동기 처리, 토큰 최적화, 모니터링 시스템 구현
"""

import os
import sys
import json
import time
import uuid
import logging
import asyncio
import hashlib
import threading
from typing import Dict, List, Any, Optional, Union, Callable
from dataclasses import dataclass, asdict, field
from datetime import datetime, timedelta
from collections import defaultdict, deque
from concurrent.futures import ThreadPoolExecutor, as_completed
from functools import wraps, lru_cache
import pickle
import sqlite3

# 외부 라이브러리
import streamlit as st
from openai import OpenAI
import redis
import psutil
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from prometheus_client import Counter, Histogram, Gauge, start_http_server
import asyncio
import aiohttp

# 로컬 모듈
sys.path.append('..')
from config import get_config

# 설정 및 로깅
config = get_config()
logger = logging.getLogger(__name__)

# Prometheus 메트릭 정의
REQUESTS_TOTAL = Counter('chatbot_requests_total', 'Total requests', ['method', 'endpoint'])
REQUEST_DURATION = Histogram('chatbot_request_duration_seconds', 'Request duration')
CACHE_HITS = Counter('chatbot_cache_hits_total', 'Cache hits')
CACHE_MISSES = Counter('chatbot_cache_misses_total', 'Cache misses')
TOKEN_USAGE = Counter('chatbot_tokens_total', 'Token usage', ['type'])
ACTIVE_CONNECTIONS = Gauge('chatbot_active_connections', 'Active connections')
ERROR_RATE = Counter('chatbot_errors_total', 'Total errors', ['error_type'])

print("환경 설정 완료!")

## 1. 캐싱 시스템

### 1.1 캐시 매니저 구현
Redis와 로컬 메모리를 활용한 다층 캐싱 시스템을 구현합니다.

In [ ]:
@dataclass
class CacheMetrics:
    hits: int = 0
    misses: int = 0
    evictions: int = 0
    total_size: int = 0
    hit_rate: float = 0.0

class CacheManager:
    def __init__(self, redis_host='localhost', redis_port=6379, 
                 redis_db=0, local_cache_size=1000):
        """
        다층 캐싱 시스템 초기화
        - L1: 로컬 메모리 캐시 (빠름)
        - L2: Redis 캐시 (영속적)
        """
        self.logger = logging.getLogger(__name__)
        
        # Redis 연결
        try:
            self.redis_client = redis.Redis(
                host=redis_host, 
                port=redis_port, 
                db=redis_db,
                decode_responses=True
            )
            self.redis_client.ping()
            self.redis_available = True
            self.logger.info("Redis 연결 성공")
        except Exception as e:
            self.redis_client = None
            self.redis_available = False
            self.logger.warning(f"Redis 연결 실패: {e}")
        
        # 로컬 캐시 (LRU)
        self.local_cache = {}
        self.local_cache_order = deque()
        self.local_cache_size = local_cache_size
        
        # 메트릭
        self.metrics = CacheMetrics()
        self.lock = threading.Lock()
    
    def _generate_cache_key(self, key_data: Dict[str, Any]) -> str:
        """
        캐시 키 생성 (해시 기반)
        """
        key_string = json.dumps(key_data, sort_keys=True)
        return hashlib.sha256(key_string.encode()).hexdigest()[:16]
    
    def get(self, key: str) -> Optional[Any]:
        """
        캐시에서 값 조회 (L1 -> L2 순서)
        """
        with self.lock:
            # L1 캐시 확인
            if key in self.local_cache:
                # LRU 업데이트
                self.local_cache_order.remove(key)
                self.local_cache_order.append(key)
                
                self.metrics.hits += 1
                CACHE_HITS.inc()
                
                self.logger.debug(f"L1 캐시 히트: {key}")
                return self.local_cache[key]
            
            # L2 캐시 확인 (Redis)
            if self.redis_available:
                try:
                    cached_value = self.redis_client.get(key)
                    if cached_value:
                        # JSON 역직렬화
                        value = json.loads(cached_value)
                        
                        # L1 캐시에도 저장
                        self._store_local(key, value)
                        
                        self.metrics.hits += 1
                        CACHE_HITS.inc()
                        
                        self.logger.debug(f"L2 캐시 히트: {key}")
                        return value
                except Exception as e:
                    self.logger.error(f"Redis 조회 오류: {e}")
            
            # 캐시 미스
            self.metrics.misses += 1
            CACHE_MISSES.inc()
            return None
    
    def set(self, key: str, value: Any, ttl: int = 3600):
        """
        캐시에 값 저장
        """
        with self.lock:
            # L1 캐시 저장
            self._store_local(key, value)
            
            # L2 캐시 저장 (Redis)
            if self.redis_available:
                try:
                    serialized_value = json.dumps(value)
                    self.redis_client.setex(key, ttl, serialized_value)
                except Exception as e:
                    self.logger.error(f"Redis 저장 오류: {e}")
    
    def _store_local(self, key: str, value: Any):
        """
        로컬 캐시에 저장 (LRU 정책)
        """
        if key in self.local_cache:
            self.local_cache_order.remove(key)
        
        self.local_cache[key] = value
        self.local_cache_order.append(key)
        
        # 캐시 크기 제한
        while len(self.local_cache) > self.local_cache_size:
            oldest_key = self.local_cache_order.popleft()
            del self.local_cache[oldest_key]
            self.metrics.evictions += 1
    
    def get_metrics(self) -> CacheMetrics:
        """
        캐시 메트릭 조회
        """
        total_requests = self.metrics.hits + self.metrics.misses
        if total_requests > 0:
            self.metrics.hit_rate = self.metrics.hits / total_requests
        
        self.metrics.total_size = len(self.local_cache)
        return self.metrics

# 캐시 매니저 테스트
cache_manager = CacheManager(local_cache_size=10)

# 테스트 데이터
test_key = "test_conversation"
test_data = {"response": "안녕하세요!", "tokens": 100}

# 저장 및 조회
cache_manager.set(test_key, test_data)
cached_result = cache_manager.get(test_key)

print(f"캐시 저장/조회 테스트: {cached_result}")
print(f"캐시 메트릭: {cache_manager.get_metrics()}")

## 2. 비동기 처리

### 2.1 비동기 챗봇 구현
여러 요청을 동시에 처리할 수 있는 비동기 챗봇을 구현합니다.

In [ ]:
class AsyncChatbot:
    def __init__(self, cache_manager: CacheManager):
        """
        비동기 챗봇 초기화
        """
        self.logger = logging.getLogger(__name__)
        self.client = OpenAI(api_key=config.get('openai_api_key'))
        self.cache_manager = cache_manager
        
        # 동시 처리 제한
        self.semaphore = asyncio.Semaphore(10)  # 최대 10개 동시 요청
        
        # 메트릭
        self.active_requests = 0
        self.total_requests = 0
        
    async def chat_completion(self, messages: List[Dict], 
                             model: str = "gpt-3.5-turbo",
                             use_cache: bool = True,
                             **kwargs) -> Dict[str, Any]:
        """
        비동기 채팅 완성 요청
        """
        async with self.semaphore:
            start_time = time.time()
            self.active_requests += 1
            self.total_requests += 1
            
            ACTIVE_CONNECTIONS.set(self.active_requests)
            REQUESTS_TOTAL.labels(method='chat', endpoint='completion').inc()
            
            try:
                # 캐시 키 생성
                cache_key = None
                if use_cache:
                    cache_data = {
                        "messages": messages,
                        "model": model,
                        **kwargs
                    }
                    cache_key = self.cache_manager._generate_cache_key(cache_data)
                    
                    # 캐시 조회
                    cached_result = self.cache_manager.get(cache_key)
                    if cached_result:
                        self.logger.info(f"캐시에서 응답 반환: {cache_key}")
                        return cached_result
                
                # OpenAI API 호출 (비동기 시뮬레이션)
                loop = asyncio.get_event_loop()
                response = await loop.run_in_executor(
                    None, 
                    self._make_openai_request,
                    messages, model, kwargs
                )
                
                # 응답 처리
                result = {
                    "response": response.choices[0].message.content,
                    "model": response.model,
                    "usage": {
                        "prompt_tokens": response.usage.prompt_tokens,
                        "completion_tokens": response.usage.completion_tokens,
                        "total_tokens": response.usage.total_tokens
                    },
                    "timestamp": datetime.now().isoformat(),
                    "duration": time.time() - start_time
                }
                
                # 토큰 사용량 기록
                TOKEN_USAGE.labels(type='prompt').inc(response.usage.prompt_tokens)
                TOKEN_USAGE.labels(type='completion').inc(response.usage.completion_tokens)
                
                # 캐시에 저장
                if use_cache and cache_key:
                    self.cache_manager.set(cache_key, result, ttl=3600)
                
                return result
                
            except Exception as e:
                ERROR_RATE.labels(error_type=type(e).__name__).inc()
                self.logger.error(f"채팅 완성 오류: {e}")
                raise
            finally:
                self.active_requests -= 1
                ACTIVE_CONNECTIONS.set(self.active_requests)
                REQUEST_DURATION.observe(time.time() - start_time)
    
    def _make_openai_request(self, messages: List[Dict], 
                           model: str, kwargs: Dict) -> Any:
        """
        동기 OpenAI API 호출 (스레드 풀에서 실행)
        """
        return self.client.chat.completions.create(
            model=model,
            messages=messages,
            **kwargs
        )
    
    async def batch_chat_completion(self, 
                                   batch_requests: List[Dict]) -> List[Dict]:
        """
        배치 처리 - 여러 요청을 동시에 처리
        """
        tasks = []
        
        for request in batch_requests:
            task = asyncio.create_task(
                self.chat_completion(**request)
            )
            tasks.append(task)
        
        # 모든 작업 완료 대기
        results = await asyncio.gather(*tasks, return_exceptions=True)
        
        # 예외 처리
        processed_results = []
        for i, result in enumerate(results):
            if isinstance(result, Exception):
                self.logger.error(f"배치 요청 {i} 실패: {result}")
                processed_results.append({
                    "error": str(result),
                    "request_index": i
                })
            else:
                processed_results.append(result)
        
        return processed_results

# 비동기 챗봇 테스트
async def test_async_chatbot():
    chatbot = AsyncChatbot(cache_manager)
    
    # 단일 요청 테스트
    messages = [{"role": "user", "content": "안녕하세요!"}]
    
    start_time = time.time()
    result = await chatbot.chat_completion(messages)
    end_time = time.time()
    
    print(f"단일 요청 결과: {result['response'][:50]}...")
    print(f"처리 시간: {end_time - start_time:.2f}초")
    
    # 배치 요청 테스트
    batch_requests = [
        {"messages": [{"role": "user", "content": "1+1은?"}]},
        {"messages": [{"role": "user", "content": "파이썬이란?"}]},
        {"messages": [{"role": "user", "content": "AI의 미래는?"}]}
    ]
    
    start_time = time.time()
    batch_results = await chatbot.batch_chat_completion(batch_requests)
    end_time = time.time()
    
    print(f"\n배치 처리 결과 ({len(batch_results)}개):")
    for i, result in enumerate(batch_results):
        if 'error' not in result:
            print(f"  {i+1}: {result['response'][:30]}...")
    
    print(f"배치 처리 시간: {end_time - start_time:.2f}초")

# 테스트 실행 (주석 해제하여 실행)
# await test_async_chatbot()

## 3. 토큰 최적화

### 3.1 토큰 최적화 관리자
토큰 사용량을 모니터링하고 최적화하는 시스템을 구현합니다.

In [ ]:
@dataclass
class TokenUsage:
    prompt_tokens: int = 0
    completion_tokens: int = 0
    total_tokens: int = 0
    cost: float = 0.0
    timestamp: datetime = field(default_factory=datetime.now)

class TokenOptimizer:
    def __init__(self):
        """
        토큰 최적화 관리자 초기화
        """
        self.logger = logging.getLogger(__name__)
        
        # 모델별 토큰 가격 (USD per 1K tokens)
        self.token_prices = {
            "gpt-3.5-turbo": {"prompt": 0.0005, "completion": 0.0015},
            "gpt-4": {"prompt": 0.01, "completion": 0.03},
            "gpt-4-turbo": {"prompt": 0.01, "completion": 0.03}
        }
        
        # 사용량 추적
        self.usage_history = deque(maxlen=10000)
        self.daily_usage = defaultdict(TokenUsage)
        
        # 최적화 설정
        self.max_context_length = {
            "gpt-3.5-turbo": 4096,
            "gpt-4": 8192,
            "gpt-4-turbo": 128000
        }
    
    def calculate_cost(self, usage: Dict[str, int], model: str) -> float:
        """
        토큰 사용량 기반 비용 계산
        """
        if model not in self.token_prices:
            self.logger.warning(f"알 수 없는 모델: {model}")
            return 0.0
        
        prices = self.token_prices[model]
        
        prompt_cost = (usage.get('prompt_tokens', 0) / 1000) * prices['prompt']
        completion_cost = (usage.get('completion_tokens', 0) / 1000) * prices['completion']
        
        return prompt_cost + completion_cost
    
    def track_usage(self, usage: Dict[str, int], model: str):
        """
        토큰 사용량 추적
        """
        cost = self.calculate_cost(usage, model)
        
        token_usage = TokenUsage(
            prompt_tokens=usage.get('prompt_tokens', 0),
            completion_tokens=usage.get('completion_tokens', 0),
            total_tokens=usage.get('total_tokens', 0),
            cost=cost
        )
        
        self.usage_history.append(token_usage)
        
        # 일별 사용량 누적
        today = datetime.now().date().isoformat()
        daily = self.daily_usage[today]
        daily.prompt_tokens += token_usage.prompt_tokens
        daily.completion_tokens += token_usage.completion_tokens
        daily.total_tokens += token_usage.total_tokens
        daily.cost += token_usage.cost
    
    def optimize_messages(self, messages: List[Dict], 
                         model: str = "gpt-3.5-turbo") -> List[Dict]:
        """
        메시지 최적화 (컨텍스트 길이 제한)
        """
        max_length = self.max_context_length.get(model, 4096)
        
        # 토큰 수 추정 (대략적)
        estimated_tokens = sum(len(msg.get('content', '').split()) * 1.3 
                              for msg in messages)
        
        if estimated_tokens <= max_length * 0.8:  # 80% 임계값
            return messages
        
        # 시스템 메시지와 최근 메시지 유지
        optimized_messages = []
        
        # 시스템 메시지 보존
        for msg in messages:
            if msg.get('role') == 'system':
                optimized_messages.append(msg)
        
        # 최근 대화 유지 (역순으로 추가)
        remaining_tokens = max_length * 0.7
        current_tokens = 0
        
        for msg in reversed(messages):
            if msg.get('role') != 'system':
                msg_tokens = len(msg.get('content', '').split()) * 1.3
                if current_tokens + msg_tokens <= remaining_tokens:
                    optimized_messages.insert(-len([m for m in optimized_messages 
                                                   if m.get('role') != 'system']), msg)
                    current_tokens += msg_tokens
                else:
                    break
        
        self.logger.info(f"메시지 최적화: {len(messages)} -> {len(optimized_messages)}")
        return optimized_messages
    
    def get_usage_summary(self, days: int = 7) -> Dict[str, Any]:
        """
        사용량 요약 정보 조회
        """
        # 최근 N일 데이터
        end_date = datetime.now().date()
        start_date = end_date - timedelta(days=days-1)
        
        period_usage = TokenUsage()
        daily_breakdown = {}
        
        current_date = start_date
        while current_date <= end_date:
            date_str = current_date.isoformat()
            daily_data = self.daily_usage.get(date_str, TokenUsage())
            
            daily_breakdown[date_str] = asdict(daily_data)
            
            period_usage.prompt_tokens += daily_data.prompt_tokens
            period_usage.completion_tokens += daily_data.completion_tokens
            period_usage.total_tokens += daily_data.total_tokens
            period_usage.cost += daily_data.cost
            
            current_date += timedelta(days=1)
        
        return {
            "period": f"{start_date} ~ {end_date}",
            "total_usage": asdict(period_usage),
            "daily_breakdown": daily_breakdown,
            "average_daily_cost": period_usage.cost / days if days > 0 else 0,
            "total_requests": len(self.usage_history)
        }

# 토큰 최적화 테스트
token_optimizer = TokenOptimizer()

# 사용량 추적 테스트
test_usage = {
    "prompt_tokens": 100,
    "completion_tokens": 50,
    "total_tokens": 150
}

token_optimizer.track_usage(test_usage, "gpt-3.5-turbo")
cost = token_optimizer.calculate_cost(test_usage, "gpt-3.5-turbo")

print(f"토큰 비용 계산: ${cost:.6f}")

# 메시지 최적화 테스트
long_messages = [
    {"role": "system", "content": "당신은 도움이 되는 AI 어시스턴트입니다."},
    {"role": "user", "content": "안녕하세요!" * 100},
    {"role": "assistant", "content": "안녕하세요! 무엇을 도와드릴까요?" * 100},
    {"role": "user", "content": "파이썬에 대해 알려주세요." * 100}
]

optimized = token_optimizer.optimize_messages(long_messages)
print(f"메시지 최적화: {len(long_messages)} -> {len(optimized)}")

# 사용량 요약
summary = token_optimizer.get_usage_summary(days=1)
print(f"사용량 요약: {summary['total_usage']}")

## 4. 모니터링 시스템

### 4.1 성능 모니터 구현
시스템 성능과 메트릭을 실시간으로 모니터링하는 시스템을 구현합니다.

In [ ]:
@dataclass
class SystemMetrics:
    cpu_percent: float = 0.0
    memory_percent: float = 0.0
    disk_usage: float = 0.0
    network_io: Dict[str, int] = field(default_factory=dict)
    timestamp: datetime = field(default_factory=datetime.now)

class PerformanceMonitor:
    def __init__(self, db_path: str = "performance_metrics.db"):
        """
        성능 모니터 초기화
        """
        self.logger = logging.getLogger(__name__)
        self.db_path = db_path
        
        # 메트릭 저장소
        self.metrics_buffer = deque(maxlen=1000)
        
        # 알림 임계값
        self.alert_thresholds = {
            "cpu_percent": 80.0,
            "memory_percent": 85.0,
            "response_time": 5.0,  # 초
            "error_rate": 0.05     # 5%
        }
        
        # 데이터베이스 초기화
        self._init_database()
        
        # 모니터링 스레드
        self.monitoring_active = False
        self.monitor_thread = None
    
    def _init_database(self):
        """
        SQLite 데이터베이스 초기화
        """
        try:
            conn = sqlite3.connect(self.db_path)
            cursor = conn.cursor()
            
            # 시스템 메트릭 테이블
            cursor.execute("""
                CREATE TABLE IF NOT EXISTS system_metrics (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    timestamp DATETIME,
                    cpu_percent REAL,
                    memory_percent REAL,
                    disk_usage REAL,
                    network_sent INTEGER,
                    network_recv INTEGER
                )
            """)
            
            # 응답 시간 메트릭 테이블
            cursor.execute("""
                CREATE TABLE IF NOT EXISTS response_metrics (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    timestamp DATETIME,
                    response_time REAL,
                    tokens_used INTEGER,
                    model TEXT,
                    cached BOOLEAN,
                    error TEXT
                )
            """)
            
            conn.commit()
            conn.close()
            
            self.logger.info("데이터베이스 초기화 완료")
        except Exception as e:
            self.logger.error(f"데이터베이스 초기화 오류: {e}")
    
    def collect_system_metrics(self) -> SystemMetrics:
        """
        시스템 메트릭 수집
        """
        try:
            # CPU 사용률
            cpu_percent = psutil.cpu_percent(interval=1)
            
            # 메모리 사용률
            memory = psutil.virtual_memory()
            memory_percent = memory.percent
            
            # 디스크 사용률
            disk = psutil.disk_usage('/')
            disk_usage = (disk.used / disk.total) * 100
            
            # 네트워크 I/O
            network = psutil.net_io_counters()
            network_io = {
                "bytes_sent": network.bytes_sent,
                "bytes_recv": network.bytes_recv
            }
            
            metrics = SystemMetrics(
                cpu_percent=cpu_percent,
                memory_percent=memory_percent,
                disk_usage=disk_usage,
                network_io=network_io
            )
            
            return metrics
            
        except Exception as e:
            self.logger.error(f"시스템 메트릭 수집 오류: {e}")
            return SystemMetrics()
    
    def store_metrics(self, metrics: SystemMetrics):
        """
        메트릭을 데이터베이스에 저장
        """
        try:
            conn = sqlite3.connect(self.db_path)
            cursor = conn.cursor()
            
            cursor.execute("""
                INSERT INTO system_metrics 
                (timestamp, cpu_percent, memory_percent, disk_usage, 
                 network_sent, network_recv)
                VALUES (?, ?, ?, ?, ?, ?)
            """, (
                metrics.timestamp.isoformat(),
                metrics.cpu_percent,
                metrics.memory_percent,
                metrics.disk_usage,
                metrics.network_io.get('bytes_sent', 0),
                metrics.network_io.get('bytes_recv', 0)
            ))
            
            conn.commit()
            conn.close()
            
        except Exception as e:
            self.logger.error(f"메트릭 저장 오류: {e}")
    
    def record_response_time(self, response_time: float, 
                           tokens_used: int = 0,
                           model: str = "",
                           cached: bool = False,
                           error: str = None):
        """
        응답 시간 메트릭 기록
        """
        try:
            conn = sqlite3.connect(self.db_path)
            cursor = conn.cursor()
            
            cursor.execute("""
                INSERT INTO response_metrics 
                (timestamp, response_time, tokens_used, model, cached, error)
                VALUES (?, ?, ?, ?, ?, ?)
            """, (
                datetime.now().isoformat(),
                response_time,
                tokens_used,
                model,
                cached,
                error
            ))
            
            conn.commit()
            conn.close()
            
        except Exception as e:
            self.logger.error(f"응답 시간 기록 오류: {e}")
    
    def check_alerts(self, metrics: SystemMetrics) -> List[str]:
        """
        알림 조건 확인
        """
        alerts = []
        
        if metrics.cpu_percent > self.alert_thresholds["cpu_percent"]:
            alerts.append(f"높은 CPU 사용률: {metrics.cpu_percent:.1f}%")
        
        if metrics.memory_percent > self.alert_thresholds["memory_percent"]:
            alerts.append(f"높은 메모리 사용률: {metrics.memory_percent:.1f}%")
        
        return alerts
    
    def start_monitoring(self, interval: int = 30):
        """
        모니터링 시작
        """
        if self.monitoring_active:
            self.logger.warning("모니터링이 이미 실행 중입니다")
            return
        
        self.monitoring_active = True
        
        def monitor_loop():
            while self.monitoring_active:
                try:
                    # 메트릭 수집
                    metrics = self.collect_system_metrics()
                    
                    # 저장
                    self.store_metrics(metrics)
                    self.metrics_buffer.append(metrics)
                    
                    # 알림 확인
                    alerts = self.check_alerts(metrics)
                    for alert in alerts:
                        self.logger.warning(f"알림: {alert}")
                    
                    time.sleep(interval)
                    
                except Exception as e:
                    self.logger.error(f"모니터링 루프 오류: {e}")
                    time.sleep(interval)
        
        self.monitor_thread = threading.Thread(target=monitor_loop)
        self.monitor_thread.daemon = True
        self.monitor_thread.start()
        
        self.logger.info(f"모니터링 시작 (간격: {interval}초)")
    
    def stop_monitoring(self):
        """
        모니터링 중지
        """
        self.monitoring_active = False
        if self.monitor_thread:
            self.monitor_thread.join(timeout=5)
        self.logger.info("모니터링 중지")
    
    def get_recent_metrics(self, hours: int = 1) -> List[SystemMetrics]:
        """
        최근 메트릭 조회
        """
        cutoff_time = datetime.now() - timedelta(hours=hours)
        return [m for m in self.metrics_buffer 
                if m.timestamp > cutoff_time]

# 성능 모니터 테스트
performance_monitor = PerformanceMonitor()

# 현재 시스템 메트릭 수집
current_metrics = performance_monitor.collect_system_metrics()
print(f"현재 시스템 상태:")
print(f"  CPU: {current_metrics.cpu_percent:.1f}%")
print(f"  메모리: {current_metrics.memory_percent:.1f}%")
print(f"  디스크: {current_metrics.disk_usage:.1f}%")

# 알림 확인
alerts = performance_monitor.check_alerts(current_metrics)
if alerts:
    print(f"알림: {alerts}")
else:
    print("시스템 정상")

# 응답 시간 기록
performance_monitor.record_response_time(1.25, tokens_used=150, 
                                       model="gpt-3.5-turbo", cached=True)
print("응답 시간 메트릭 기록 완료")

## 5. 성능 분석

### 5.1 성능 분석 도구
수집된 메트릭을 분석하고 시각화하는 도구를 구현합니다.

In [ ]:
class PerformanceAnalyzer:
    def __init__(self, monitor: PerformanceMonitor):
        """
        성능 분석기 초기화
        """
        self.monitor = monitor
        self.logger = logging.getLogger(__name__)
    
    def analyze_response_times(self, hours: int = 24) -> Dict[str, Any]:
        """
        응답 시간 분석
        """
        try:
            conn = sqlite3.connect(self.monitor.db_path)
            
            # 최근 N시간 데이터 조회
            query = """
                SELECT response_time, tokens_used, model, cached, error
                FROM response_metrics
                WHERE timestamp >= datetime('now', '-{} hours')
                ORDER BY timestamp DESC
            """.format(hours)
            
            df = pd.read_sql_query(query, conn)
            conn.close()
            
            if df.empty:
                return {"message": "분석할 데이터가 없습니다"}
            
            # 기본 통계
            response_times = df['response_time']
            analysis = {
                "total_requests": len(df),
                "avg_response_time": response_times.mean(),
                "median_response_time": response_times.median(),
                "p95_response_time": response_times.quantile(0.95),
                "p99_response_time": response_times.quantile(0.99),
                "min_response_time": response_times.min(),
                "max_response_time": response_times.max()
            }
            
            # 캐시 히트율
            cache_hits = df['cached'].sum()
            analysis['cache_hit_rate'] = cache_hits / len(df)
            
            # 에러율
            errors = df['error'].notna().sum()
            analysis['error_rate'] = errors / len(df)
            
            # 모델별 분석
            model_analysis = {}
            for model in df['model'].unique():
                model_data = df[df['model'] == model]
                model_analysis[model] = {
                    "count": len(model_data),
                    "avg_response_time": model_data['response_time'].mean(),
                    "avg_tokens": model_data['tokens_used'].mean()
                }
            
            analysis['by_model'] = model_analysis
            
            return analysis
            
        except Exception as e:
            self.logger.error(f"응답 시간 분석 오류: {e}")
            return {"error": str(e)}
    
    def analyze_system_performance(self, hours: int = 24) -> Dict[str, Any]:
        """
        시스템 성능 분석
        """
        try:
            conn = sqlite3.connect(self.monitor.db_path)
            
            query = """
                SELECT timestamp, cpu_percent, memory_percent, disk_usage
                FROM system_metrics
                WHERE timestamp >= datetime('now', '-{} hours')
                ORDER BY timestamp
            """.format(hours)
            
            df = pd.read_sql_query(query, conn)
            conn.close()
            
            if df.empty:
                return {"message": "분석할 데이터가 없습니다"}
            
            analysis = {
                "cpu": {
                    "avg": df['cpu_percent'].mean(),
                    "max": df['cpu_percent'].max(),
                    "min": df['cpu_percent'].min(),
                    "std": df['cpu_percent'].std()
                },
                "memory": {
                    "avg": df['memory_percent'].mean(),
                    "max": df['memory_percent'].max(),
                    "min": df['memory_percent'].min(),
                    "std": df['memory_percent'].std()
                },
                "disk": {
                    "current": df['disk_usage'].iloc[-1],
                    "trend": "increasing" if df['disk_usage'].iloc[-1] > df['disk_usage'].iloc[0] else "stable"
                }
            }
            
            return analysis
            
        except Exception as e:
            self.logger.error(f"시스템 성능 분석 오류: {e}")
            return {"error": str(e)}
    
    def generate_performance_report(self) -> str:
        """
        성능 리포트 생성
        """
        response_analysis = self.analyze_response_times()
        system_analysis = self.analyze_system_performance()
        
        report = []
        report.append("# 성능 분석 리포트")
        report.append(f"생성 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        report.append("")
        
        # 응답 시간 분석
        if "error" not in response_analysis:
            report.append("## 응답 시간 분석")
            report.append(f"- 총 요청 수: {response_analysis.get('total_requests', 0):,}")
            report.append(f"- 평균 응답시간: {response_analysis.get('avg_response_time', 0):.3f}초")
            report.append(f"- 95% 응답시간: {response_analysis.get('p95_response_time', 0):.3f}초")
            report.append(f"- 캐시 히트율: {response_analysis.get('cache_hit_rate', 0):.1%}")
            report.append(f"- 에러율: {response_analysis.get('error_rate', 0):.1%}")
            report.append("")
        
        # 시스템 성능 분석
        if "error" not in system_analysis:
            report.append("## 시스템 리소스 분석")
            cpu = system_analysis.get('cpu', {})
            memory = system_analysis.get('memory', {})
            
            report.append(f"- CPU 평균 사용률: {cpu.get('avg', 0):.1f}%")
            report.append(f"- CPU 최대 사용률: {cpu.get('max', 0):.1f}%")
            report.append(f"- 메모리 평균 사용률: {memory.get('avg', 0):.1f}%")
            report.append(f"- 메모리 최대 사용률: {memory.get('max', 0):.1f}%")
            report.append("")
        
        # 권장사항
        report.append("## 권장사항")
        
        if response_analysis.get('cache_hit_rate', 0) < 0.5:
            report.append("- 캐시 히트율이 낮습니다. 캐싱 전략을 검토하세요.")
        
        if response_analysis.get('avg_response_time', 0) > 3.0:
            report.append("- 평균 응답시간이 높습니다. 최적화를 고려하세요.")
        
        if system_analysis.get('cpu', {}).get('avg', 0) > 70:
            report.append("- CPU 사용률이 높습니다. 스케일링을 검토하세요.")
        
        if system_analysis.get('memory', {}).get('avg', 0) > 80:
            report.append("- 메모리 사용률이 높습니다. 메모리 최적화가 필요합니다.")
        
        return "\n".join(report)
    
    def plot_performance_charts(self):
        """
        성능 차트 생성
        """
        try:
            conn = sqlite3.connect(self.monitor.db_path)
            
            # 응답 시간 데이터
            response_df = pd.read_sql_query("""
                SELECT timestamp, response_time, cached
                FROM response_metrics
                WHERE timestamp >= datetime('now', '-24 hours')
                ORDER BY timestamp
            """, conn)
            
            # 시스템 메트릭 데이터
            system_df = pd.read_sql_query("""
                SELECT timestamp, cpu_percent, memory_percent
                FROM system_metrics
                WHERE timestamp >= datetime('now', '-24 hours')
                ORDER BY timestamp
            """, conn)
            
            conn.close()
            
            # 차트 생성
            fig, axes = plt.subplots(2, 2, figsize=(15, 10))
            
            # 응답 시간 분포
            if not response_df.empty:
                axes[0, 0].hist(response_df['response_time'], bins=30, alpha=0.7)
                axes[0, 0].set_title('응답 시간 분포')
                axes[0, 0].set_xlabel('응답 시간 (초)')
                axes[0, 0].set_ylabel('빈도')
            
            # 캐시 히트 비율
            if not response_df.empty:
                cache_counts = response_df['cached'].value_counts()
                axes[0, 1].pie(cache_counts.values, labels=['캐시 미스', '캐시 히트'], 
                             autopct='%1.1f%%')
                axes[0, 1].set_title('캐시 히트율')
            
            # CPU 사용률 시계열
            if not system_df.empty:
                system_df['timestamp'] = pd.to_datetime(system_df['timestamp'])
                axes[1, 0].plot(system_df['timestamp'], system_df['cpu_percent'])
                axes[1, 0].set_title('CPU 사용률 추이')
                axes[1, 0].set_ylabel('CPU %')
                axes[1, 0].tick_params(axis='x', rotation=45)
            
            # 메모리 사용률 시계열
            if not system_df.empty:
                axes[1, 1].plot(system_df['timestamp'], system_df['memory_percent'], 
                              color='orange')
                axes[1, 1].set_title('메모리 사용률 추이')
                axes[1, 1].set_ylabel('메모리 %')
                axes[1, 1].tick_params(axis='x', rotation=45)
            
            plt.tight_layout()
            return fig
            
        except Exception as e:
            self.logger.error(f"차트 생성 오류: {e}")
            return None

# 성능 분석기 테스트
analyzer = PerformanceAnalyzer(performance_monitor)

# 몇 개의 샘플 데이터 추가
for i in range(5):
    performance_monitor.record_response_time(
        response_time=np.random.normal(2.0, 0.5),
        tokens_used=np.random.randint(50, 200),
        model="gpt-3.5-turbo",
        cached=np.random.choice([True, False])
    )

# 분석 실행
response_analysis = analyzer.analyze_response_times(hours=1)
print("응답 시간 분석 결과:")
for key, value in response_analysis.items():
    if isinstance(value, dict):
        print(f"  {key}: {value}")
    else:
        print(f"  {key}: {value}")

# 성능 리포트 생성
print("\n" + "="*50)
report = analyzer.generate_performance_report()
print(report)

## 6. Streamlit 대시보드

### 6.1 실시간 모니터링 대시보드
성능 메트릭을 실시간으로 시각화하는 웹 대시보드를 구현합니다.

In [ ]:
def create_monitoring_dashboard():
    """
    Streamlit 모니터링 대시보드 생성
    """
    st.set_page_config(
        page_title="AI 챗봇 성능 모니터링",
        page_icon="📊",
        layout="wide"
    )
    
    st.title("🚀 AI 챗봇 성능 모니터링 대시보드")
    
    # 사이드바 설정
    st.sidebar.header("설정")
    
    # 모니터링 컴포넌트 초기화
    if 'cache_manager' not in st.session_state:
        st.session_state.cache_manager = CacheManager()
    
    if 'performance_monitor' not in st.session_state:
        st.session_state.performance_monitor = PerformanceMonitor()
    
    if 'analyzer' not in st.session_state:
        st.session_state.analyzer = PerformanceAnalyzer(
            st.session_state.performance_monitor
        )
    
    if 'chatbot' not in st.session_state:
        st.session_state.chatbot = AsyncChatbot(
            st.session_state.cache_manager
        )
    
    # 실시간 메트릭 표시
    col1, col2, col3, col4 = st.columns(4)
    
    # 현재 시스템 상태
    current_metrics = st.session_state.performance_monitor.collect_system_metrics()
    
    with col1:
        st.metric(
            label="CPU 사용률",
            value=f"{current_metrics.cpu_percent:.1f}%",
            delta=None
        )
    
    with col2:
        st.metric(
            label="메모리 사용률",
            value=f"{current_metrics.memory_percent:.1f}%",
            delta=None
        )
    
    with col3:
        cache_metrics = st.session_state.cache_manager.get_metrics()
        st.metric(
            label="캐시 히트율",
            value=f"{cache_metrics.hit_rate:.1%}",
            delta=None
        )
    
    with col4:
        st.metric(
            label="활성 연결",
            value=f"{st.session_state.chatbot.active_requests}",
            delta=None
        )
    
    # 탭 구성
    tab1, tab2, tab3, tab4 = st.tabs(["💬 챗봇 테스트", "📈 성능 분석", "🔧 캐시 관리", "📊 시스템 모니터링"])
    
    with tab1:
        st.header("챗봇 성능 테스트")
        
        # 챗봇 설정
        col1, col2 = st.columns(2)
        
        with col1:
            model = st.selectbox(
                "모델 선택",
                ["gpt-3.5-turbo", "gpt-4", "gpt-4-turbo"]
            )
        
        with col2:
            use_cache = st.checkbox("캐시 사용", value=True)
        
        # 채팅 인터페이스
        user_input = st.text_area("메시지 입력", height=100)
        
        if st.button("전송", type="primary"):
            if user_input.strip():
                with st.spinner("응답 생성 중..."):
                    start_time = time.time()
                    
                    try:
                        # 비동기 함수를 동기적으로 실행
                        import asyncio
                        
                        # 새로운 이벤트 루프 생성 (Streamlit 환경)
                        try:
                            loop = asyncio.get_event_loop()
                        except RuntimeError:
                            loop = asyncio.new_event_loop()
                            asyncio.set_event_loop(loop)
                        
                        messages = [{"role": "user", "content": user_input}]
                        result = loop.run_until_complete(
                            st.session_state.chatbot.chat_completion(
                                messages, model=model, use_cache=use_cache
                            )
                        )
                        
                        response_time = time.time() - start_time
                        
                        # 결과 표시
                        st.success("응답 완료!")
                        
                        col1, col2 = st.columns([3, 1])
                        
                        with col1:
                            st.text_area(
                                "AI 응답",
                                value=result['response'],
                                height=200,
                                disabled=True
                            )
                        
                        with col2:
                            st.write("**메트릭**")
                            st.write(f"응답시간: {result['duration']:.2f}초")
                            st.write(f"토큰 사용: {result['usage']['total_tokens']}")
                            st.write(f"모델: {result['model']}")
                        
                        # 성능 메트릭 기록
                        st.session_state.performance_monitor.record_response_time(
                            response_time=result['duration'],
                            tokens_used=result['usage']['total_tokens'],
                            model=result['model'],
                            cached=False  # 실제 구현에서는 캐시 여부 확인
                        )
                        
                    except Exception as e:
                        st.error(f"오류 발생: {e}")
                        
                        # 에러 메트릭 기록
                        st.session_state.performance_monitor.record_response_time(
                            response_time=time.time() - start_time,
                            error=str(e)
                        )
    
    with tab2:
        st.header("성능 분석")
        
        # 분석 기간 선택
        analysis_hours = st.selectbox(
            "분석 기간",
            [1, 6, 12, 24, 48, 72],
            index=3
        )
        
        if st.button("분석 실행"):
            with st.spinner("분석 중..."):
                # 응답 시간 분석
                response_analysis = st.session_state.analyzer.analyze_response_times(
                    hours=analysis_hours
                )
                
                if "error" not in response_analysis:
                    col1, col2 = st.columns(2)
                    
                    with col1:
                        st.subheader("응답 시간 통계")
                        
                        metrics_data = {
                            "메트릭": ["평균", "중앙값", "95%", "99%", "최소", "최대"],
                            "값 (초)": [
                                f"{response_analysis.get('avg_response_time', 0):.3f}",
                                f"{response_analysis.get('median_response_time', 0):.3f}",
                                f"{response_analysis.get('p95_response_time', 0):.3f}",
                                f"{response_analysis.get('p99_response_time', 0):.3f}",
                                f"{response_analysis.get('min_response_time', 0):.3f}",
                                f"{response_analysis.get('max_response_time', 0):.3f}"
                            ]
                        }
                        
                        st.table(pd.DataFrame(metrics_data))
                    
                    with col2:
                        st.subheader("성능 지표")
                        
                        st.metric(
                            "총 요청 수",
                            f"{response_analysis.get('total_requests', 0):,}"
                        )
                        
                        st.metric(
                            "캐시 히트율",
                            f"{response_analysis.get('cache_hit_rate', 0):.1%}"
                        )
                        
                        st.metric(
                            "에러율",
                            f"{response_analysis.get('error_rate', 0):.1%}"
                        )
                    
                    # 모델별 분석
                    if response_analysis.get('by_model'):
                        st.subheader("모델별 성능")
                        
                        model_data = []
                        for model, stats in response_analysis['by_model'].items():
                            model_data.append({
                                "모델": model,
                                "요청 수": stats['count'],
                                "평균 응답시간": f"{stats['avg_response_time']:.3f}초",
                                "평균 토큰": f"{stats['avg_tokens']:.0f}"
                            })
                        
                        st.table(pd.DataFrame(model_data))
                
                else:
                    st.warning("분석할 데이터가 충분하지 않습니다.")
        
        # 성능 리포트
        if st.button("성능 리포트 생성"):
            report = st.session_state.analyzer.generate_performance_report()
            st.text_area("성능 리포트", value=report, height=400)
    
    with tab3:
        st.header("캐시 관리")
        
        # 캐시 메트릭
        cache_metrics = st.session_state.cache_manager.get_metrics()
        
        col1, col2, col3 = st.columns(3)
        
        with col1:
            st.metric("히트 수", cache_metrics.hits)
        
        with col2:
            st.metric("미스 수", cache_metrics.misses)
        
        with col3:
            st.metric("캐시 크기", cache_metrics.total_size)
        
        # 캐시 히트율 차트
        if cache_metrics.hits + cache_metrics.misses > 0:
            hit_rate_data = pd.DataFrame({
                "상태": ["히트", "미스"],
                "횟수": [cache_metrics.hits, cache_metrics.misses]
            })
            
            st.subheader("캐시 히트율")
            st.bar_chart(hit_rate_data.set_index("상태"))
        
        # 캐시 관리 기능
        st.subheader("캐시 관리")
        
        col1, col2 = st.columns(2)
        
        with col1:
            if st.button("캐시 통계 새로고침"):
                st.experimental_rerun()
        
        with col2:
            if st.button("로컬 캐시 클리어"):
                st.session_state.cache_manager.local_cache.clear()
                st.session_state.cache_manager.local_cache_order.clear()
                st.success("로컬 캐시가 클리어되었습니다.")
    
    with tab4:
        st.header("시스템 모니터링")
        
        # 실시간 시스템 메트릭
        current_metrics = st.session_state.performance_monitor.collect_system_metrics()
        
        col1, col2 = st.columns(2)
        
        with col1:
            st.subheader("CPU 및 메모리")
            
            # CPU 사용률 게이지
            cpu_color = "red" if current_metrics.cpu_percent > 80 else "orange" if current_metrics.cpu_percent > 60 else "green"
            st.metric("CPU 사용률", f"{current_metrics.cpu_percent:.1f}%")
            st.progress(current_metrics.cpu_percent / 100)
            
            # 메모리 사용률 게이지
            memory_color = "red" if current_metrics.memory_percent > 80 else "orange" if current_metrics.memory_percent > 60 else "green"
            st.metric("메모리 사용률", f"{current_metrics.memory_percent:.1f}%")
            st.progress(current_metrics.memory_percent / 100)
        
        with col2:
            st.subheader("디스크 및 네트워크")
            
            st.metric("디스크 사용률", f"{current_metrics.disk_usage:.1f}%")
            
            if current_metrics.network_io:
                st.metric(
                    "네트워크 송신",
                    f"{current_metrics.network_io.get('bytes_sent', 0) / 1024 / 1024:.1f} MB"
                )
                st.metric(
                    "네트워크 수신",
                    f"{current_metrics.network_io.get('bytes_recv', 0) / 1024 / 1024:.1f} MB"
                )
        
        # 알림 확인
        alerts = st.session_state.performance_monitor.check_alerts(current_metrics)
        if alerts:
            st.warning("⚠️ 시스템 알림")
            for alert in alerts:
                st.write(f"• {alert}")
        else:
            st.success("✅ 시스템 정상")
        
        # 모니터링 제어
        st.subheader("모니터링 제어")
        
        col1, col2 = st.columns(2)
        
        with col1:
            monitoring_interval = st.slider(
                "모니터링 간격 (초)",
                min_value=5,
                max_value=300,
                value=30
            )
        
        with col2:
            if not st.session_state.performance_monitor.monitoring_active:
                if st.button("모니터링 시작", type="primary"):
                    st.session_state.performance_monitor.start_monitoring(
                        interval=monitoring_interval
                    )
                    st.success("모니터링이 시작되었습니다.")
            else:
                if st.button("모니터링 중지", type="secondary"):
                    st.session_state.performance_monitor.stop_monitoring()
                    st.info("모니터링이 중지되었습니다.")
    
    # 자동 새로고침
    if st.sidebar.checkbox("자동 새로고침 (10초)"):
        time.sleep(10)
        st.experimental_rerun()

# 대시보드 실행 안내
st.markdown("""
### 🚀 Streamlit 대시보드 실행 방법

위의 코드를 별도 파일 (`monitoring_dashboard.py`)로 저장한 후, 다음 명령어로 실행하세요:

```bash
streamlit run monitoring_dashboard.py
```

대시보드에서 제공하는 기능:
- **실시간 시스템 모니터링**: CPU, 메모리, 디스크 사용률
- **챗봇 성능 테스트**: 다양한 모델로 응답 성능 테스트
- **캐시 관리**: 캐시 히트율 및 관리 기능
- **성능 분석**: 상세한 성능 메트릭 분석 및 리포트
""")

print("Streamlit 대시보드 코드 생성 완료!")

## 7. 실습 과제

### 과제 1: 고급 캐싱 전략 구현
1. **LFU(Least Frequently Used) 캐시 정책 구현**
   - 사용 빈도를 추적하는 캐싱 시스템
   - 메모리 효율성 비교 분석

2. **분산 캐싱 시스템 구현**
   - 여러 Redis 인스턴스를 활용한 샤딩
   - 일관성 해싱을 통한 부하 분산

### 과제 2: 비동기 성능 최적화
1. **커넥션 풀링 구현**
   - OpenAI API 연결을 위한 커넥션 풀
   - 동시 요청 처리 성능 향상

2. **백프레셔(Backpressure) 제어**
   - 과부하 상황에서의 요청 제어
   - 큐잉 시스템 구현

### 과제 3: 모니터링 고도화
1. **분산 트레이싱 구현**
   - OpenTelemetry를 사용한 요청 추적
   - 마이크로서비스 간 호출 추적

2. **실시간 알림 시스템**
   - Slack/Discord 웹훅 연동
   - 임계값 기반 자동 알림

### 과제 4: 성능 벤치마킹
1. **부하 테스트 도구 개발**
   - 동시 사용자 시뮬레이션
   - 성능 메트릭 수집 및 분석

2. **A/B 테스트 프레임워크**
   - 다양한 최적화 기법 비교
   - 통계적 유의성 검증

### 제출 방법
```bash
# 과제 디렉토리 생성
mkdir lesson6_assignments
cd lesson6_assignments

# 각 과제별 구현 파일
touch advanced_caching.py
touch async_optimization.py
touch monitoring_advanced.py
touch performance_benchmark.py

# 실행 결과 보고서
touch performance_report.md
```

## 📋 학습 정리

### 배운 내용
1. **캐싱 시스템**: 다층 캐싱을 통한 응답 속도 향상
2. **비동기 처리**: 동시 요청 처리를 통한 처리량 증대
3. **토큰 최적화**: 비용 효율적인 AI 모델 사용
4. **모니터링**: 실시간 성능 추적 및 분석
5. **대시보드**: 시각적 성능 모니터링 도구

### 핵심 개념
- **성능 메트릭**: 응답시간, 처리량, 에러율, 리소스 사용률
- **캐싱 전략**: LRU, 다층 캐싱, TTL 관리
- **비동기 패턴**: 세마포어, 배치 처리, 커넥션 풀
- **모니터링**: 메트릭 수집, 알림, 대시보드

### 다음 단계
다음 차시에서는 **배포 및 통합**을 다룹니다:
- Docker 컨테이너화
- FastAPI REST API 구현
- CI/CD 파이프라인 구축
- 프로덕션 환경 배포